<font size=6.5 color='blue'>Discovering Groups</font>

In [3]:
import pandas as pd
import numpy as np

In [5]:
blogdata=pd.read_table('blogdata.txt')

# 分级聚类

读入数据

In [11]:
def readflie(filename):
    file = open(filename,'r')
    lines = [line for line in file]
    
    colnames=lines[0].strip().split('\t')[1:]
    rownames=[]
    data=[]
    for line in lines[1:]:
        p = line.strip().split('\t')
        rownames.append(p[0])
        data.append([float(x) for x in p[1:]])
    return rownames,colnames,data

紧密度计算
- `1-pearsonr`

In [14]:
# from math import sqrt
# def pearson(v1,v2):
#     sum1=sum(v1)
#     sum2=sum(v2)
    
#     sum1sq=sum([pow(x,2) for x in v1])
#     sum2sq=sum([pow(x,2) for x in v2])
    
#     psum= sum([v1[i]*v2[i] for i in range(len(v1))])
    
#     num=psum-(sum1*sum2/len(v1))
#     den = den=sqrt((sum1Sq-pow(sum1,2)/len(v1))*(sum2Sq-pow(sum2,2)/len(v1)))
#     if den==0: return 0
    
#     return 1.0-num/den

from scipy.stats import pearsonr
def pearson(v1,v2):
    r,p_value = pearsonr(v1,v2)
    if p_value==1: r=0
    return 1.0 - r

In [15]:
class bicluster:
    def __init__(self,vec,left=None,right=None,distance=0.0, i_d=None):
        self.left=left
        self.right=right
        self.vec=vec
        self.i_d=i_d
        self.distance = distance

In [19]:
blognames,words,data=readflie('blogdata.txt')

In [96]:
def hcluster(rows,distance=pearson):
    distances={}
    currentclustid=-1
    
    clust = [bicluster(rows[i],i_d=i) for i in range(len(rows))]
    
    while len(clust)>1:
        lowestpair = (0,1)
        closest=distance(clust[0].vec, clust[1].vec)

        for i in range(len(clust)):
            for j in range(i+1,len(clust)):
                if (clust[i].i_d,clust[j].i_d) not in distances:
                    distances[(clust[i].i_d,clust[j].i_d)]=distance(clust[i].vec,clust[j].vec)
                d = distances[(clust[i].i_d,clust[j].i_d)]

                if d<closest:
                    closest=d
                    lowestpair=(i,j)
        mergevec=[
            (clust[lowestpair[0]].vec[i] + clust[lowestpair[1]].vec[i]) /2.0 
            for i in range(len(clust[0].vec))]

        newcluster = bicluster(mergevec,left=clust[lowestpair[0]], right=clust[lowestpair[1]],distance=closest, i_d=currentclustid)   
        #构件新的 bicluster 对象，并将上级两个 bicluster 元素写入 left/right属性

        currentclustid-=1
        del clust[lowestpair[1]]  #先删除大序号，再删除小序号，否则序号会乱
        del clust[lowestpair[0]]
        clust.append(newcluster)  #i_d为负数，且放在clust最后
    
    return clust[0]

In [40]:
top_clust = hcluster(data)

In [97]:
def printclust(clust,labels=None, n=0):
    for i in range(n):
        print(' ',end='')
    if clust.i_d<0:
        print('-')
    else:
        if labels==None: print(clust.i_d)
        else: print(labels[clust.i_d])     
    
    #递归搜索        
    if clust.left!=None: printclust(clust.left,labels=labels, n=n+1)
    if clust.right!=None: printclust(clust.right,labels=labels, n=n+1)

In [64]:
printclust(top_clust,labels=blognames)

-
 gapingvoid: "cartoons drawn on the back of business cards"
 -
  -
   Schneier on Security
   Instapundit.com
  -
   The Blotter
   -
    -
     MetaFilter
     -
      SpikedHumor
      -
       Captain's Quarters
       -
        Michelle Malkin
        -
         -
          NewsBusters.org - Exposing Liberal Media Bias
          -
           -
            Hot Air
            Crooks and Liars
           -
            Power Line
            Think Progress
         -
          Andrew Sullivan | The Daily Dish
          -
           Little Green Footballs
           -
            Eschaton
            -
             Talking Points Memo: by Joshua Micah Marshall
             Daily Kos
    -
     43 Folders
     -
      TechEBlog
      -
       -
        Mashable!
        Signum sine tinnitu--by Guy Kawasaki
       -
        -
         -
          Slashdot
          -
           MAKE Magazine
           Boing Boing
         -
          -
           Oilman
           -
            Online

In [18]:
from PIL import Image, ImageDraw